In [46]:
#########################################################################
## COMPROBAR GPU ASIGNADA EN COLABORATORY
#########################################################################
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar  4 10:37:15 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   40C    P8    17W / 100W |   1333MiB /  6144MiB |     41%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [47]:
!pip install imitation

In [48]:
!pip install seals

In [24]:
!pip install imitation[test]

In [49]:
#########################################################################
## LIBRERIAS NECESARIAS
#########################################################################
import tensorflow as tf
import gymnasium as gym
import sinergym 
from sinergym.utils.wrappers import (LoggerWrapper, NormalizeAction,
                                     NormalizeObservation) 
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import keras


# Librerias necesarias para BC
from stable_baselines3.common.evaluation import evaluate_policy

from imitation.algorithms import bc
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
# Problema en rollout: es la función que define las transiciones expertas
import imitation.data.rollout as rollout 
from stable_baselines3.common.vec_env import DummyVecEnv


# **5Zone**

## 1. Creamos un vector de entornos

In [63]:
def _make_env():
     _env = gym.make("Eplus-5zone-hot-discrete-v1")
     _env = NormalizeObservation(_env)
     _env = LoggerWrapper(_env)
     #_env = RolloutInfoWrapper(_env)
     return _env

venv = DummyVecEnv([_make_env for _ in range(1)])

#==============================================================================================#
#==============================================================================================#


## 2.Creación de demostraciones expertas 

In [64]:
# Función que devuekve la política experta 
def download_expert():
    print("Downloading a pretrained expert.")
    expert = load_policy(
        "ppo",
        path="model5zone.zip",
        venv=venv,
    )
    return expert

# Función que devuelve trayectorias de la política experta 
def sample_expert_transitions():
    # Cargamos la política experta
    expert = download_expert()

    print("Sampling expert transitions.")

    # Generar trayectorias a partir de una política dada
    rollouts = rollout.rollout(
        expert,  # Política 
        venv,    # Entorno
        sample_until=rollout.make_sample_until(min_timesteps=None, min_episodes=1),  # EPISODES=1 asi que min_episodes=1
        rng=np.random.default_rng(),
        unwrap=False,
    )
    
    return rollout.flatten_trajectories(rollouts)


In [65]:
# Selección de una muestra de trayectorias de secuencias expertas
transitions = sample_expert_transitions()

Sampling expert transitions.
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


# **ALGORITMOS DE IMITATION LEARNING**

## 1. **Behavioral Cloning (BC)**

In [66]:
import torch

# Check if GPU is available and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Definición  

In [71]:
# Definición BC
bc_trainer = bc.BC(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    demonstrations=transitions,
    rng=np.random.default_rng(),
    batch_size=28032,
)


### Dudas----> SOLUCIONADAS :)
* Las tuplas de observaciones no son soportadas porque no están en un formato numpy array 
* ¿Como cambio el formato si Sinergym utiliza estas observaciones?

In [58]:
# from stable_baselines3.common.monitor import Monitor
# Assuming `env` is your evaluation environment
#eval_env = Monitor(venv)

### Entrenamiento y evaluación

In [72]:
reward_before_training, _ = evaluate_policy(bc_trainer.policy, venv, 1)
print(f"Reward before training: {reward_before_training}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(a

Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Reward before training: -22992.83762663603


In [73]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [74]:
import subprocess

# Ejecuta el comando pip para actualizar una biblioteca
# Reemplaza 'nombre_biblioteca' con el nombre de la biblioteca que deseas actualizar
subprocess.call(['pip', 'install', '--upgrade', 'imitation'])

0

In [75]:
bc_trainer.train(n_epochs=100, progress_bar=True)

0batch [00:00, ?batch/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA_gather)

In [ ]:
# Cambiar el 10 por los episodios que quiera evaluar, en GAIL y RI-GAIL evaluaba en 50 episodios
reward, _ = evaluate_policy(bc_trainer.policy, venv, 1)
print("Reward:", reward)

In [59]:
from imitation.algorithms.dagger import SimpleDAggerTrainer
import tempfile

In [56]:
rng=np.random.default_rng(0)

### Definición y entrenamiento 

In [57]:
bc_trainer = bc.BC(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    rng=rng,
)

In [61]:
with tempfile.TemporaryDirectory(prefix="dagger_example_") as tmpdir:
    print(tmpdir)
    dagger_trainer = SimpleDAggerTrainer(
        venv=venv,
        scratch_dir=tmpdir,
        expert_policy=download_expert(),
        bc_trainer=bc_trainer,
        rng=rng,
    )
    dagger_trainer.train(8_000)


/tmp/dagger_example_i_f7i8y7


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

0batch [00:00, ?batch/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA_gather)

### Evaluación 

In [ ]:
# Cambiar el 10 por los episodios que quiera evaluar, en GAIL y RI-GAIL evaluaba en 50 episodios
reward, _ = evaluate_policy(dagger_trainer.policy, venv, 50)
print("Reward:", reward)

## 3. **Adversarial Inverse Reinforcement Learning (AIRL)**

* Similar a GAIL
* Se diferencia en que cubre casos donde la función de recompensa es ,as generalizable a cambios dinámicos del entorno

In [62]:
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from imitation.algorithms.adversarial.airl import AIRL
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy


In [63]:
SEED = 42

learner = PPO(
    env=venv,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0005,
    gamma=0.95,
    clip_range=0.1,
    vf_coef=0.1,
    n_epochs=5,
    seed=SEED,
)

reward_net = BasicShapedRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)
airl_trainer = AIRL(
    demonstrations=transitions,
    demo_batch_size=2048,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=16,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


In [68]:
venv.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, venv, 1, return_episode_rewards=True,
)

airl_trainer.train(2048)  # Train for 2_000_000 steps to match expert.
venv.seed(SEED)

learner_rewards_after_training, _ = evaluate_policy(
    learner, venv, 1, return_episode_rewards=True,
)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(a

Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


round:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
--------------------------------------------------------------------------------------------------------------| 6%
| raw/                        |      |
|    gen/time/fps             | 334  |
|    gen/time/iterations      | 1    |
|    gen/time/time_elapsed    | 6    |
|    gen/time/total_timesteps | 2048 |
--------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.5      |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0        |
|    disc/disc_entropy                | 0.354    |
|    disc/disc_loss              

round: 100%|██████████| 1/1 [00:06<00:00,  6.79s/it]


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


In [69]:
print("mean reward after training:", np.mean(learner_rewards_after_training))
print("mean reward before training:", np.mean(learner_rewards_before_training))


mean reward after training: -25222.75435392186
mean reward before training: -22276.52855842607


## 4. **Density-based reward modeling**
* Es una técnica de IRL que asigna recompensas más altas a estados o pares estado-acción que ocurren con más frecuencia en las demostraciones de un experto 
* Esta variante utiliza la estimación de densidad del núcleo para modelar la distribución subyacente de las demostraciones de expertos. Asigna recompensas a estados o pares estado-acción según su log-verosimilitud estimada bajo la distribución de las demostraciones de expertos.
*  La intuición clave detrás de este método es incentivar al agente a tomar acciones que se asemejen a las acciones del experto en estados similares.

In [76]:
from stable_baselines3.common.policies import ActorCriticPolicy
from imitation.algorithms import density as db
import pprint

In [71]:
imitation_trainer = PPO(
    ActorCriticPolicy, venv, learning_rate=3e-4, gamma=0.95, ent_coef=1e-4, n_steps=2048
)

In [72]:
density_trainer = db.DensityAlgorithm(
    venv=venv,
    rng=rng,
    demonstrations=transitions,
    rl_algo=imitation_trainer,
    density_type=db.DetrayectoriesnsityType.STATE_ACTION_DENSITY,
    is_stationary=True,
    kernel="gaussian",
    kernel_bandwidth=0.4,
    standardise_inputs=True,
)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


In [73]:
density_trainer.train()

In [77]:
def print_stats(density_trainer, n_trajectories):
    stats = density_trainer.test_policy(n_trajectories=n_trajectories)
    print("True reward function stats:")
    pprint.pprint(stats)
    stats_im = density_trainer.test_policy(true_reward=False, n_trajectories=n_trajectories)
    print("Imitation reward function stats:")
    pprint.pprint(stats_im)

In [78]:
print("Stats before training:")
print_stats(density_trainer, 1)

density_trainer.train_policy(100)  # Train for 1_000_000 steps to approach expert performance.

print("Stats after training:")
print_stats(density_trainer, 1)

Stats before training:


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
True reward function stats:
{'len_max': 35040,
 'len_mean': 35040.0,
 'len_min': 35040,
 'len_std': 0.0,
 'n_traj': 1,
 'return_max': -23182.00149248261,
 'return_mean': -23182.00149248261,
 'return_min': -23182.00149248261,
 'return_std': 0.0}


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Imitation reward function stats:
{'len_max': 35040,
 'len_mean': 35040.0,
 'len_min': 35040,
 'len_std': 0.0,
 'n_traj': 1,
 'return_max': -945346.3150596619,
 'return_mean': -945346.3150596619,
 'return_min': -945346.3150596619,
 'return_std': 0.0}


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Stats after training:-----------------------------------------------------------------------------------------| 6%
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#---------------------------------------------

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Imitation reward function stats:
{'len_max': 35040,
 'len_mean': 35040.0,
 'len_min': 35040,
 'len_std': 0.0,
 'n_traj': 1,
 'return_max': -943862.0244865417,
 'return_mean': -943862.0244865417,
 'return_min': -943862.0244865417,
 'return_std': 0.0}


## 5. **Maximum Causal Entropy Inverse Reinforcement Learning (MCE IRL)**

* Este algoritmo implementa una interacción con el modelo haciendo uso del principio de entropía causal máxima

In [85]:
from imitation.algorithms.mce_irl import (
    MCEIRL,
    mce_occupancy_measures,
    mce_partition_fh,
)
 
from imitation.rewards import reward_nets


from functools import partial

In [80]:
# Mirar https://imitation.readthedocs.io/en/latest/algorithms/mce_irl.html pporque dice que venv solo es un entorno vectorizado
# No se si necesito volver a crear venv o no
reward_net = reward_nets.BasicRewardNet(
    venv.observation_space,
    venv.action_space,
    hid_sizes=[256],
    use_action=False,
    use_done=False,
    use_next_state=False,
)

In [89]:
env = gym.make("Eplus-5zone-hot-discrete-v1")
env = NormalizeObservation(env)
env = LoggerWrapper(env)

#==============================================================================================#
#==============================================================================================#


In [93]:
env_creator = partial(env, height=4, horizon=8, width=7, use_xy_obs=True)
env_single = env_creator()

state_env_creator = lambda: base_envs.ExposePOMDPStateWrapper(env_creator())
trayectories
# This is just a vectorized environment because `generate_trajectories` expects one
state_venv = DummyVecEnv([state_env_creator] * 4)

_, _, pi = mce_partition_fh(env_single)

_, om = mce_occupancy_measures(env_single, pi=pi)

TypeError: the first argument must be callable

In [81]:
mce_irl = MCEIRL(
    om,
    venv,#env_single, DUDA
    reward_net,
    log_interval=250,
    optimizer_kwargs={"lr": 0.01},
    rng=rng,
)

NameError: name 'om' is not defined

In [ ]:
occ_measure = mce_irl.train()

In [ ]:
imitation_trajs = rollout.generate_trajectories(
    policy=mce_irl.policy,
    venv=venv, # state_venv, 
    sample_until=rollout.make_min_timesteps(5000),
    rng=rng,
)

In [ ]:
print("Imitation stats: ", rollout.rollout_stats(imitation_trajs)) 

## 6. **Reward Learning through Preference Comparisons**
* El algoritmo de comparación de preferencias aprende una función de recompensa a partir de las preferencias entre pares de trayectorias. Las comparaciones se modelan como generadas a partir de un modelo de Bradley-Terry (o Boltzmann racional), donde la probabilidad de preferir la trayectoria A sobre B es proporcional al exponencial de la diferencia entre el retorno de la trayectoria A menos B. 
* En otras palabras, la diferencia en los retornos forma un logit para un problema de clasificación binaria, y en consecuencia la función de recompensa se entrena utilizando una pérdida de entropía cruzada para predecir la comparación de preferencias.






In [94]:
from imitation.rewards.reward_nets import BasicRewardNet 
from imitation.algorithms import preference_comparisons
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
from imitation.util.networks import RunningNorm

In [95]:
reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm,
)

In [96]:
fragmenter = preference_comparisons.RandomFragmenter(warning_threshold=0, rng=rng)

gatherer = preference_comparisons.SyntheticGatherer(rng=rng)

preference_model = preference_comparisons.PreferenceModel(reward_net)

reward_trainer = preference_comparisons.BasicRewardTrainer(
    preference_model=preference_model,
    loss=preference_comparisons.CrossEntropyRewardLoss(),
    epochs=10,
    rng=rng,
)

In [97]:
agent = PPO(
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    n_steps=2048 // venv.num_envs,
    clip_range=0.1,
    ent_coef=0.01,
    gae_lambda=0.95,
    n_epochs=10,
    gamma=0.97,
    learning_rate=2e-3,
)

In [98]:
trajectory_generator = preference_comparisons.AgentTrainer(
    algorithm=agent,
    reward_fn=reward_net,
    venv=venv,
    exploration_frac=0.05,
    rng=rng,
)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

Progress: |****************************************************************************************************| 100%


In [ ]:
pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    num_iterations=5, # Set to 60 for better performance
    fragmenter=fragmenter,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    initial_epoch_multiplier=4,
    initial_comparison_frac=0.1,
    query_schedule="hyperbolic",
)

In [ ]:
pref_comparisons.train(total_timesteps=50_000, total_comparisons=200)

In [ ]:
n_eval_episodes = 50
reward_mean, reward_std = evaluate_policy(agent.policy, venv, n_eval_episodes)
reward_stderr = reward_std/np.sqrt(n_eval_episodes)

print(f"Reward: {reward_mean:.0f} +/- {reward_stderr:.0f}")

## 7. **Soft Q Imitation Learning (SQIL)**
* El aprendizaje de imitación Soft Q aprende a imitar una política a partir de demostraciones utilizando el algoritmo DQN con recompensas modificadas. 
* Durante cada actualización de la política, la mitad del lote se muestrea de las demostraciones y la otra mitad se muestrea del entorno. Las demostraciones de expertos se les asigna una recompensa de 1, y al entorno se le asigna una recompensa de 0. Esto anima a la política a imitar las demostraciones y, al mismo tiempo, a evitar estados no vistos en las demostraciones.

In [99]:
from imitation.algorithms import sqil

In [101]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [102]:
# Hint: set to 1_000_000 to match the expert performance.
sqil_trainer.train(total_timesteps=1_000)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


In [104]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 1)
print("Reward:", reward)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Reward: -25133.39557491988
